In [182]:
import pandas as pd
import numpy as np

In [183]:
#read each of the .csv tables
fl_equ = pd.read_csv('FL_opioid_equ.csv')
ga_equ = pd.read_csv('GA_opioid_equ.csv')
counties = pd.read_csv('Counties_DF_Merge_Ready_v2.csv')
deaths = pd.read_csv('overdose_deaths.csv')
controls = pd.read_csv('State_opioid_Controls_Year_Changed.csv')

In [184]:
#Dekalb was a problem county, changed name in counties to be without a space
counties = counties.replace(to_replace = 'DE KALB', value = 'DEKALB')

In [185]:
counties['State'].value_counts().sum()

6783

In [186]:
deaths.sample(20)

,index,State,County,Year,Deaths
2871,427,OK,Cleveland County,2008.0,30.0
7440,314,MI,Genesee County,2015.0,117.0
2416,544,WA,Whatcom County,2007.0,12.0
6000,331,MO,St. Louis city,2013.0,85.0
5119,132,GA,Bartow County,2012.0,13.0
5421,434,NY,Richmond County,2012.0,77.0
3482,441,OK,Wagoner County,2009.0,14.0
1352,9,AL,Tuscaloosa County,2006.0,22.0
1271,409,TX,Brazoria County,2005.0,31.0
16,16,CA,El Dorado County,2003.0,10.0


In [187]:
#merge the counties and population data with the control states table
outer_counties = controls.merge(counties, how = 'outer', on = 'State', indicator = True )

In [188]:
outer_counties.head()
#outer_counties.sample(50)
#outer_counties.describe()
#outer_counties._merge.value_counts()

,State_Full,Year_Changed,State,Unnamed: 0,County,FIPS,Year,Population,_merge
0,California,Control,CA,183,ALAMEDA,6001,2006,1444484.0,both
1,California,Control,CA,184,ALPINE,6003,2006,1276.0,both
2,California,Control,CA,185,AMADOR,6005,2006,38353.0,both
3,California,Control,CA,186,BUTTE,6007,2006,216824.0,both
4,California,Control,CA,187,CALAVERAS,6009,2006,46045.0,both


In [189]:
#outer_counties['Year'].dtypes

In [190]:
#Clean the death data table to replace County, Borough, Parish, and city with '' and make them upper case
deaths['County'] = deaths['County'].replace(regex = True, to_replace = [' County', ' Borough', ' Parish', ' City'], value = '')
deaths['County'] = deaths['County'].replace(regex = True, to_replace = ['St. '], value = 'Saint ')
deaths['County'] = deaths['County'].str.upper()
deaths['Year'] = deaths['Year'].astype(int)
#Confirmed the changes were correct

In [191]:
deaths.sample(10)
#outer_counties.sample(10)

,index,State,County,Year,Deaths
3780,140,GA,DOUGLAS,2010,10.0
7882,756,WI,JEFFERSON,2015,10.0
3413,372,NM,RIO ARRIBA,2009,19.0
1654,311,NJ,MONMOUTH,2006,70.0
6994,619,TN,LOUDON,2014,10.0
4978,685,WV,LOGAN,2011,24.0
7729,603,PA,SUSQUEHANNA,2015,10.0
36,36,CA,SAN MATEO,2003,51.0
5383,396,NJ,PASSAIC,2012,46.0
6803,428,NM,SAN JUAN,2014,22.0


In [192]:
#drop indicator column
outer_counties = outer_counties.drop(axis = 1, columns = '_merge')

In [193]:
death_added = deaths.merge(outer_counties, how = 'outer', on = ['State', 'Year', 'County'], indicator = True)
death_added._merge.value_counts()
#death_added.sample(10)
#This resulted in less left_only joins than expected, but this is because not all counties have death data for drug overdoses

left_only     6451
right_only    5314
both          1469
Name: _merge, dtype: int64

In [194]:
death_added['Year'].value_counts().sum()

13234

In [195]:
death_added.sample(20)

,index,State,County,Year,Deaths,State_Full,Year_Changed,Unnamed: 0,FIPS,Population,_merge
13013,NaN,MO,LACLEDE,2012,NaN,Montana,Control,20389.0,29105.0,35507.0,right_only
6749,374.0,NC,HENDERSON,2014,11.0,NaN,NaN,NaN,NaN,NaN,left_only
5316,329.0,MO,SAINT CHARLES,2012,37.0,Montana,Control,20428.0,29183.0,360695.0,both
6924,549.0,PA,CHESTER,2014,68.0,NaN,NaN,NaN,NaN,NaN,left_only
4247,607.0,VA,VIRGINIA BEACH CITY,2010,14.0,NaN,NaN,NaN,NaN,NaN,left_only
2200,328.0,NJ,CAMDEN,2007,97.0,NaN,NaN,NaN,NaN,NaN,left_only
3613,572.0,WA,GRANT,2009,17.0,Washington,2012,12392.0,53025.0,83612.0,both
5689,20.0,AL,TUSCALOOSA,2013,30.0,NaN,NaN,NaN,NaN,NaN,left_only
12573,NaN,MO,ATCHISON,2008,NaN,Montana,Control,7767.0,29005.0,5798.0,right_only
8044,NaN,CA,COLUSA,2012,NaN,California,Control,19046.0,6011.0,21329.0,right_only


In [196]:
#rename columns to match with the intermediate merged table for Florida
fl_equ = fl_equ.rename(columns = {'BUYER_COUNTY':'County', 'County_FIPS':'FIPS'})
fl_equ.head()

,Year,FIPS,County,Morphine_equ,State
0,2006,12001,ALACHUA,82596.618688,FL
1,2006,12003,BAKER,11900.178638,FL
2,2006,12005,BAY,61998.102472,FL
3,2006,12007,BRADFORD,13424.830895,FL
4,2006,12009,BREVARD,201186.153403,FL


In [197]:
#rename columns to match with the intermediate merged table for Georgia
ga_equ = ga_equ.rename(columns = {'BUYER_COUNTY':'County', 'County_FIPS':'FIPS'})
ga_equ.head()

,Year,FIPS,County,Morphine_equ,State
0,2006,13001,APPLING,3996.843779,GA
1,2006,13003,ATKINSON,1375.483686,GA
2,2006,13005,BACON,4586.778211,GA
3,2006,13007,BAKER,9.651600,GA
4,2006,13009,BALDWIN,11069.705010,GA


In [198]:
#drop indicator column from death_added
death_added = death_added.drop(axis = 1, columns = '_merge')

In [199]:
#merge florida morphine and georgia morphine
#Expect only left or right merges
m_combined = ga_equ.merge(fl_equ, how = 'outer', on = ['County', 'State', 'Morphine_equ','FIPS', 'Year'], indicator = True)
m_combined.sample(10)

,Year,FIPS,County,Morphine_equ,State,_merge
257,2007,13217,NEWTON,18236.840235,GA,left_only
774,2011,13031,BULLOCH,19002.412651,GA,left_only
333,2008,13061,CLAY,753.027578,GA,left_only
1195,2008,12001,ALACHUA,114675.221922,FL,right_only
175,2007,13047,CATOOSA,22704.154371,GA,left_only
397,2008,13191,MCINTOSH,4607.686163,GA,left_only
274,2007,13253,SEMINOLE,2490.734244,GA,left_only
1374,2010,12093,OKEECHOBEE,55324.330290,FL,right_only
1074,2006,12023,COLUMBIA,27550.681709,FL,right_only
1327,2009,12133,WASHINGTON,13060.632695,FL,right_only


In [200]:
m_combined.head()

,Year,FIPS,County,Morphine_equ,State,_merge
0,2006,13001,APPLING,3996.843779,GA,left_only
1,2006,13003,ATKINSON,1375.483686,GA,left_only
2,2006,13005,BACON,4586.778211,GA,left_only
3,2006,13007,BAKER,9.651600,GA,left_only
4,2006,13009,BALDWIN,11069.705010,GA,left_only


In [201]:
death_added.sample(10)

,index,State,County,Year,Deaths,State_Full,Year_Changed,Unnamed: 0,FIPS,Population
11966,NaN,OR,COLUMBIA,2009,NaN,Oregon,Control,11637.0,41009.0,48612.0
5339,352.0,NC,FORSYTH,2012,39.0,NaN,NaN,NaN,NaN,NaN
9901,NaN,GA,PEACH,2006,NaN,Georgia,Control,494.0,13225.0,25510.0
9661,NaN,TX,GILLESPIE,2012,NaN,Texas,2007,21462.0,48171.0,24777.0
10897,NaN,AL,MONTGOMERY,2008,NaN,Alabama,Control,6336.0,1101.0,227461.0
12429,NaN,MO,NODAWAY,2006,NaN,Montana,Control,1552.0,29147.0,22698.0
8103,NaN,AZ,COCONINO,2012,NaN,Arizona,Control,18954.0,4005.0,134011.0
12588,NaN,MO,CEDAR,2008,NaN,Montana,Control,7784.0,29039.0,14120.0
1572,229.0,MI,GENESEE,2006,90.0,NaN,NaN,NaN,NaN,NaN
11563,NaN,MS,TALLAHATCHIE,2011,NaN,Mississippi,Control,17179.0,28135.0,15279.0


In [202]:
#check merge
#Validated, similar to expected counts
m_combined._merge.value_counts()

left_only     1063
right_only     466
both             0
Name: _merge, dtype: int64

In [203]:
#drop _merge column
m_combined = m_combined.drop(axis = 1, columns = '_merge')

In [204]:
#add the morphine table into the intermediate table
morphine_added = m_combined.merge(death_added, how = 'outer', on = ['County', 'State', 'FIPS','Year'], indicator = True)
#morphine_added.describe()
morphine_added._merge.value_counts()
#morphine_added.loc[:, 'Year'].value_counts()

right_only    11719
both           1515
left_only        14
Name: _merge, dtype: int64

In [205]:
#We are not going to use Georgia Anyway for morphine checks
morphine_added_check = morphine_added.loc[((morphine_added['County'] == 'MUSCOGEE'))]
morphine_added_check.head()

,Year,FIPS,County,Morphine_equ,State,index,Deaths,State_Full,Year_Changed,Unnamed: 0,Population,_merge
103,2006,13217.0,MUSCOGEE,45567.906151,GA,NaN,NaN,NaN,NaN,NaN,NaN,left_only
256,2007,13217.0,MUSCOGEE,54511.314974,GA,NaN,NaN,NaN,NaN,NaN,NaN,left_only
408,2008,13217.0,MUSCOGEE,59565.322243,GA,NaN,NaN,NaN,NaN,NaN,NaN,left_only
559,2009,13217.0,MUSCOGEE,67729.253544,GA,NaN,NaN,NaN,NaN,NaN,NaN,left_only
710,2010,13217.0,MUSCOGEE,81272.511431,GA,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [206]:
morphine_added[morphine_added._merge == 'left_only']

,Year,FIPS,County,Morphine_equ,State,index,Deaths,State_Full,Year_Changed,Unnamed: 0,Population,_merge
103,2006,13217.0,MUSCOGEE,45567.906151,GA,NaN,NaN,NaN,NaN,NaN,NaN,left_only
117,2006,13247.0,RICHMOND,44785.937567,GA,NaN,NaN,NaN,NaN,NaN,NaN,left_only
256,2007,13217.0,MUSCOGEE,54511.314974,GA,NaN,NaN,NaN,NaN,NaN,NaN,left_only
270,2007,13247.0,RICHMOND,52950.708223,GA,NaN,NaN,NaN,NaN,NaN,NaN,left_only
408,2008,13217.0,MUSCOGEE,59565.322243,GA,NaN,NaN,NaN,NaN,NaN,NaN,left_only
422,2008,13247.0,RICHMOND,61776.855617,GA,NaN,NaN,NaN,NaN,NaN,NaN,left_only
559,2009,13217.0,MUSCOGEE,67729.253544,GA,NaN,NaN,NaN,NaN,NaN,NaN,left_only
573,2009,13247.0,RICHMOND,67375.713238,GA,NaN,NaN,NaN,NaN,NaN,NaN,left_only
710,2010,13217.0,MUSCOGEE,81272.511431,GA,NaN,NaN,NaN,NaN,NaN,NaN,left_only
724,2010,13247.0,RICHMOND,81258.435288,GA,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [207]:
morphine_added.head()

,Year,FIPS,County,Morphine_equ,State,index,Deaths,State_Full,Year_Changed,Unnamed: 0,Population,_merge
0,2006,13001.0,APPLING,3996.843779,GA,NaN,NaN,Georgia,Control,384.0,17725.0,both
1,2006,13003.0,ATKINSON,1375.483686,GA,NaN,NaN,Georgia,Control,385.0,8197.0,both
2,2006,13005.0,BACON,4586.778211,GA,NaN,NaN,Georgia,Control,386.0,10682.0,both
3,2006,13007.0,BAKER,9.651600,GA,NaN,NaN,Georgia,Control,387.0,3877.0,both
4,2006,13009.0,BALDWIN,11069.705010,GA,NaN,NaN,Georgia,Control,388.0,46513.0,both


In [208]:
morphine_added._merge.value_counts()

right_only    11719
both           1515
left_only        14
Name: _merge, dtype: int64

In [209]:
#remove merge column
morphine_added = morphine_added.drop(axis = 1, columns = '_merge')

In [210]:
#Drop the unnamed and index column, which was an index column left over from the import of an intermediate table
final_frame = morphine_added.drop(columns = 'Unnamed: 0')
final_frame = final_frame.drop(columns = 'index')
final_frame.sample(10)

,Year,FIPS,County,Morphine_equ,State,Deaths,State_Full,Year_Changed,Population
8595,2015,NaN,SAINT LOUIS,NaN,MO,161.0,NaN,NaN,NaN
8270,2015,NaN,COCONINO,NaN,AZ,15.0,NaN,NaN,NaN
2617,2005,NaN,ANOKA,NaN,MN,14.0,NaN,NaN,NaN
8129,2014,NaN,COLLIN,NaN,TX,70.0,NaN,NaN,NaN
8712,2015,NaN,NIAGARA,NaN,NY,48.0,NaN,NaN,NaN
3525,2007,NaN,CALVERT,NaN,MD,13.0,NaN,NaN,NaN
13118,2007,53031.0,JEFFERSON,NaN,WA,NaN,Washington,2012,29346.0
3270,2006,48027.0,BELL,NaN,TX,10.0,Texas,2007,273771.0
8500,2015,NaN,WHITLEY,NaN,KY,16.0,NaN,NaN,NaN
2003,2004,NaN,SUSSEX,NaN,DE,11.0,NaN,NaN,NaN


In [211]:
#add in a deaths per capita column
final_frame['Deaths_PC'] = final_frame['Deaths'] / final_frame['Population']
final_frame.sample(10)

,Year,FIPS,County,Morphine_equ,State,Deaths,State_Full,Year_Changed,Population,Deaths_PC
8208,2014,NaN,BROWN,NaN,WI,20.0,NaN,NaN,NaN,NaN
6031,2011,NaN,RICHLAND,NaN,SC,36.0,NaN,NaN,NaN,NaN
2876,2006,1073.0,JEFFERSON,NaN,AL,86.0,Alabama,Control,655893.0,0.000131
7478,2013,NaN,JEFFERSON,NaN,WV,16.0,NaN,NaN,NaN,NaN
990,2012,13167.0,JOHNSON,2295.079658,GA,NaN,Georgia,Control,9981.0,NaN
339,2008,13073.0,COLUMBIA,30271.488339,GA,NaN,Georgia,Control,117504.0,NaN
8161,2014,NaN,DAVIS,NaN,UT,43.0,NaN,NaN,NaN,NaN
3521,2007,NaN,WORCESTER,NaN,MA,102.0,NaN,NaN,NaN,NaN
11319,2012,1055.0,ETOWAH,NaN,AL,NaN,Alabama,Control,104317.0,NaN
10783,2012,48183.0,GREGG,NaN,TX,NaN,Texas,2007,121451.0,NaN


In [212]:
final_frame.loc[:,'Year'].value_counts()
final_frame.loc[:,'County'].value_counts()

JEFFERSON                 148
WASHINGTON                142
MONTGOMERY                 99
MADISON                    94
FRANKLIN                   92
                         ... 
SAINT JOHN THE BAPTIST      1
HALIFAX                     1
HOCKING                     1
CATTARAUGUS                 1
LEWIS AND CLARK             1
Name: County, Length: 1160, dtype: int64

In [217]:
#export the final frame to a csv
#final_frame.to_csv('Final_Data_Frame_Python_Midterm.csv')

In [214]:
#Check filter to see if it is reasonable

In [228]:
ultimate_final_frame = final_frame.loc[(final_frame['State'] == 'CA') | (final_frame['State'] == 'AZ') | (final_frame['State'] == 'NM') | (final_frame['State'] == 'TX') | (final_frame['State'] == 'GA') | (final_frame['State'] == 'AL') | (final_frame['State'] == 'MS') | (final_frame['State'] == 'FL') | (final_frame['State'] == 'OR')| (final_frame['State'] == 'ID') | (final_frame['State'] == 'MO') | (final_frame['State'] == 'WA')]
ultimate_final_frame.sample(10)

,Year,FIPS,County,Morphine_equ,State,Deaths,State_Full,Year_Changed,Population,Deaths_PC
5589,2011,6037.0,LOS ANGELES,NaN,CA,706.0,California,Control,9787747.0,0.000072
5479,2010,48485.0,WICHITA,NaN,TX,16.0,Texas,2007,130976.0,0.000122
709,2010,13213.0,MURRAY,15191.540337,GA,10.0,Georgia,Control,39925.0,0.000250
9120,2009,6103.0,TEHAMA,NaN,CA,NaN,California,Control,60601.0,NaN
1033,2012,13257.0,STEPHENS,30549.977038,GA,NaN,Georgia,Control,26003.0,NaN
281,2007,13271.0,TELFAIR,5755.375163,GA,NaN,Georgia,Control,15932.0,NaN
12463,2006,29195.0,SALINE,NaN,MO,NaN,Montana,Control,23008.0,NaN
9163,2012,6021.0,GLENN,NaN,CA,NaN,California,Control,28078.0,NaN
11763,2011,28069.0,KEMPER,NaN,MS,NaN,Mississippi,Control,10394.0,NaN
9415,2006,48061.0,CAMERON,NaN,TX,NaN,Texas,2007,380169.0,NaN


In [229]:
ultimate_final_frame.to_csv('ultimate_final_frame.csv')